In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("joebeachcapital/sega-games")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'sega-games' dataset.
Path to dataset files: /kaggle/input/sega-games


In [ ]:
import pandas as pd

df = pd.read_csv(path + "/SegaGames.csv")
df.head()

,meta_score,title,platform,date,user_score,link,esrb_rating,developers,genres
0,NaN,Persona 3 Reload,XONE,"Feb 2, 2024",NaN,/game/xbox-one/persona-3-reload,NaN,['P-Studio'],"['Role-Playing', 'Japanese-Style']"
1,NaN,Persona 5 Tactica,PC,"Nov 17, 2023",NaN,/game/pc/persona-5-tactica,NaN,['P-Studio'],"['Strategy', 'Turn-Based', 'Tactics']"
2,NaN,Persona 5 Tactica,XONE,"Nov 17, 2023",NaN,/game/xbox-one/persona-5-tactica,RP,['P-Studio'],"['Strategy', 'Turn-Based', 'Tactics']"
3,NaN,Persona 5 Tactica,PS4,"Nov 17, 2023",NaN,/game/playstation-4/persona-5-tactica,NaN,['P-Studio'],"['Strategy', 'Turn-Based', 'Tactics']"
4,NaN,Persona 5 Tactica,Switch,"Nov 17, 2023",NaN,/game/switch/persona-5-tactica,RP,['P-Studio'],"['Strategy', 'Turn-Based', 'Tactics']"


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1481 entries, 0 to 1480
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   meta_score   788 non-null    float64
 1   title        1481 non-null   object 
 2   platform     1481 non-null   object 
 3   date         1481 non-null   object 
 4   user_score   929 non-null    float64
 5   link         1481 non-null   object 
 6   esrb_rating  1120 non-null   object 
 7   developers   1471 non-null   object 
 8   genres       1479 non-null   object 
dtypes: float64(2), object(7)
memory usage: 104.3+ KB


In [ ]:
import pandas as pd
import os

# --- 0. Setup File Path ---
# The dataset was downloaded to /kaggle/input/sega-games
path = "/kaggle/input/sega-games"
file_name = os.path.join(path, "SegaGames.csv")

print(f"Attempting to load file from: {file_name}")

# --- 1. Load Data ---
try:
    # Use the file_name variable to load the data
    # NEW: Added 'dtype_backend='pyarrow'' for Pandas 3.0+ performance
    df = pd.read_csv(file_name, dtype_backend='pyarrow')
    print("--- 1. Initial File Loaded (using PyArrow backend) ---")

except FileNotFoundError:
    print(f"Error: File not found at {file_name}")
    print("Please make sure 'SegaGames.csv' is in the correct folder.")
    # No exit() here, as the problem is with the path in this cell, not file absence.
    # If df was not loaded, subsequent steps would fail.
    exit()
except Exception as e:
    # Fallback for older pandas versions that don't support pyarrow
    if 'dtype_backend' in str(e):
        print("PyArrow backend failed (you might have an older Pandas version).")
        print("Falling back to standard loader...")
        df = pd.read_csv(file_name)
    else:
        print(f"An error occurred: {e}")
        exit()

# --- 2. Base Cleaning (All our previous steps) ---
# Replaced `inplace=True` with DataFrame reassignment
df = df.dropna(subset=['developers', 'genres'])

# Replaced `inplace=True` with column reassignment
df['esrb_rating'] = df['esrb_rating'].fillna('Not Rated')

# Fill 'meta_score' and 'user_score'
mean_meta_score = df['meta_score'].mean()
mean_user_score = df['user_score'].mean()
# Replaced `inplace=True`
df['meta_score'] = df['meta_score'].fillna(mean_meta_score)
df['user_score'] = df['user_score'].fillna(mean_user_score)

# Convert 'date', turning any errors into 'NaT' (Not a Time)
df['date'] = pd.to_datetime(df['date'], errors='coerce')

# Now drop any rows that failed the date conversion
# Replaced `inplace=True`
df = df.dropna(subset=['date'])
print(f"--- 2. Base cleaning complete. Working with {len(df)} rows. ---")


# --- 3. Platform Cleaning (Your new rules) ---
# (This logic was already modern, no changes needed)

# First, fix the specific names *before* grouping
df['platform'] = df['platform'].replace({
    'Genesis)': 'Genesis',
    'Arcade)': 'Arcade',
    'DC': 'Dreamcast'  # Change DC to Dreamcast
})

# Now, define the list of platforms we want to keep
platforms_to_keep = [
    'PC', 'iOS', 'PS3', 'X360', 'WII', 'PS4', 'Switch', 'XONE',
    'PS2', 'XBOX', 'Dreamcast', 'DS', '3DS', 'PSP', 'GC', 'GBA', 'VITA', 'WIIU',
    'Genesis', 'Arcade'
]

# Group all other platforms into an 'Other' category
df['platform'] = df['platform'].apply(lambda x: x if x in platforms_to_keep else 'Other')
print("--- 3. Platform column cleaned. ---")


# --- 4. Developer & Genre Cleaning ---
# (This logic was already modern, no changes needed)
# These columns are strings that *look* like lists: "['Sega']"
# We'll use .str accessors to clean them up by removing the brackets and quotes

# Note: .astype('string') is a good way to ensure .str accessor works
df['developers'] = df['developers'].astype('string').str.strip("[]").str.replace("'", "")
df['genres'] = df['genres'].astype('string').str.strip("[]").str.replace("'", "")
print("--- 4. Developer and Genre column text (brackets/quotes) cleaned. ---")


# --- 5. Grouping the "Long Tail" ---
# (This logic was already modern, no changes needed)
# Your Power BI charts will look messy with 200+ developers.
# Let's group any developer/genre with less than 5 games into "Other".

# Group Developers
dev_counts = df['developers'].value_counts()
developers_to_keep = dev_counts[dev_counts >= 5].index.tolist()
df['developers'] = df['developers'].apply(lambda x: x if x in developers_to_keep else 'Other')

# Group Genres
genre_counts = df['genres'].value_counts()
genres_to_keep = genre_counts[genre_counts >= 5].index.tolist()
df['genres'] = df['genres'].apply(lambda x: x if x in genres_to_keep else 'Other')
print("--- 5. 'Long tail' developers and genres grouped into 'Other'. ---")


# --- 6. Final Drop ---
# (This logic was already modern, no changes needed)
if 'link' in df.columns:
    df = df.drop(columns=['link'])
    print("--- 6. 'link' column dropped. ---")


# --- 7. FINAL STEP: Save to a new CSV file ---
output_filename = 'SegaGames_CLEAN.csv'
df.to_csv(output_filename, index=False)

print("\n" + "="*30)
print(f"  SUCCESS!  ")
print(f"Your 100% clean data is saved as '{output_filename}'")
print("You can now import *this* file into Power BI.")
print("="*30 + "\n")

# --- 8. Final Check (So you can see the results) ---
print("--- Final 'platform' Counts ---")
print(df['platform'].value_counts())

Attempting to load file from: /kaggle/input/sega-games/SegaGames.csv
--- 1. Initial File Loaded (using PyArrow backend) ---
--- 2. Base cleaning complete. Working with 1402 rows. ---
--- 3. Platform column cleaned. ---
--- 4. Developer and Genre column text (brackets/quotes) cleaned. ---
--- 5. 'Long tail' developers and genres grouped into 'Other'. ---
--- 6. 'link' column dropped. ---

  SUCCESS!  
Your 100% clean data is saved as 'SegaGames_CLEAN.csv'
You can now import *this* file into Power BI.

--- Final 'platform' Counts ---
platform
PC           391
iOS          119
X360         110
WII          110
PS3          107
PS4           94
Switch        84
XONE          77
PS2           63
XBOX          41
Dreamcast     36
DS            33
3DS           30
GC            30
PSP           26
Other         14
GBA           12
VITA          11
Genesis        7
WIIU           4
Arcade         3
Name: count, dtype: int64
